In [30]:
%%capture
%run 01_transaction.ipynb
%run 02_merkle.ipynb
%run 03_account.ipynb 

In [31]:
from objsize import get_deep_size as get_size

## Block Header

The `Header` is a blocks summary. It only stores the `root` node of the txs merkle tree, which makes it small and constant in size. The actual mining happens on the block header.

In [32]:
class Header:
    def __init__(self, root, prev_hash, number, n_txs):
        self.root      = root
        self.prev_hash = prev_hash
        self.number    = number
        self.n_txs     = n_txs
        self.mined     = False
        self.time      = time.time()
        
    def __setattr__(self, prop, val):
        super().__setattr__(prop, val)
        super().__setattr__('hash', sha(self.__dict__))
        
    def __str__(self): 
        header_str = ('hash:\t\t'     +ph(self.hash)+
                      '\ntime:\t\t'   +time.ctime(self.time)+
                      '\nnumber:\t\t' +str(self.number)+
                      '\nprev_hash:\t'+ph(self.prev_hash)+
                      '\ntxs_count:\t'+str(self.n_txs)+
                      '\ntxs_root:\t' +ph(self.root))
        if self.mined: 
            header_str += ('\nminer:\t\t' +ph(self.miner)+
                           '\ndiff:\t\t'  +str(self.diff)+
                           '\nreward:\t\t'+str(self.reward)+
                           '\nnonce:\t\t' +str(self.nonce)) 
        return header_str
    def __bytes__(self): return (self.root+
                                 self.prev_hash+
                                 str(self.number)+
                                 str(self.time)).encode()
    def __eq__(self, other): return self.hash == other.hash

In [33]:
bh = Header(rh(),rh(),2,10); print(bh)

hash:		💵 0x4f6cad7418...c87
time:		Tue Mar 30 16:58:12 2021
number:		2
prev_hash:	📴 0x8e816722d1...086
txs_count:	10
txs_root:	📪 0x84160be9c5...600


In [34]:
print(stringfy(bh))

root:          📪 0x84160be9c5...600
hash:          💵 0x4f6cad7418...c87
prev_hash:     📴 0x8e816722d1...086
number:        2 ether
n_txs:         10 ether
mined:         False
time:          Tue Mar 30 16:58:12 2021


If anything in the block header changes its `hash` changes automatically as well.

In [79]:
bh_changed        = deepcopy(tx2)
bh_changed.number = 120
assert bh != bh_changed

## Block

Consists of the block header `bh` and a list of txs. To validate that the txs are correct, the merkle tree can be rebuild and checked against the root hash stored in the `bh`.

In [80]:
class Block: 
    def __init__(self, bh, txs): 
        self.bh = bh
        self.mt = MerkleTree(txs)
        self.val_txs(txs)
        self.txs_value, self.fees = 
        
    def val_txs(self, txs):
        for tx in txs: assert val_sig(tx),   'tx signature invalid'
        assert self.are_unique(txs),         'txs include duplicates'
        assert self.mt.root == self.bh.root, 'txs root hash do not match'
    
    def are_unique(self, txs): return len(set([tx.hash for tx in txs])) == len(txs)
    def __eq__(self, other):   return super().__eq__()
    def __str__(self):         return str(self.bh) 

Create random list of signed txs.

In [81]:
acc1,acc2 = Account(),Account()
def r_stxs(n): return [acc1.signed_tx(acc2,ri(1,9),ri(1,9)/100) for _ in range(n)]

Create block from a block header containing its txs merkle tree.

In [90]:
txs = r_stxs(10)
mt  = MerkleTree(txs)
bh  = Header(mt.root, rh(), 2, len(txs))
b   = Block(bh, txs); print(b)

hash:		🔣 bd708ec7999534fa...228
time:		Mon Mar 29 21:08:16 2021
number:		2
prev_hash:	📉 63fc3336ed110470...cf8
txs_count:	10
txs_root:	🕞 f8d2194b1239240f...c87


Blocks can only contain unique txs. 

In [88]:
txs[0] = txs[1]

mt  = MerkleTree(txs)
bh  = Header(mt.root, rh(), 2, len(txs))

In [89]:
dup_detected = False
try:    Block(bh, txs)
except: dup_detected = True
assert  dup_detected